# Volatility Forecasting

This setup code is required to run in an IPython notebook

In [7]:
import warnings
warnings.simplefilter('ignore')

%matplotlib inline
import seaborn
seaborn.set_style('darkgrid')

## Data

These examples make use of S&P 500 data from Yahoo! using the pandas-datareader package to manage data download.

In [13]:
import datetime as dt
import sys

import numpy as np
import pandas as pd
import pandas_datareader.data as web

from arch import arch_model
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import acf,pacf,plot_acf,plot_pacf
from statsmodels.tsa.arima_model import ARMA

start = dt.datetime(2000,1,1)
end = dt.datetime(2017,1,1)
data = web.get_data_famafrench('F-F_Research_Data_Factors_daily', start=start, end=end)
mkt_returns = data[0]['Mkt-RF'] +  data[0]['RF']
returns = mkt_returns

In [10]:
t=sm.tsa.stattools.adfuller(returns, )
output=pd.DataFrame(index=['Test Statistic Value', "p-value", "Lags Used", "Number of Observations Used","Critical Value(1%)","Critical Value(5%)","Critical Value(10%)"],columns=['value'])
output['value']['Test Statistic Value'] = t[0]
output['value']['p-value'] = t[1]
output['value']['Lags Used'] = t[2]
output['value']['Number of Observations Used'] = t[3]
output['value']['Critical Value(1%)'] = t[4]['1%']
output['value']['Critical Value(5%)'] = t[4]['5%']
output['value']['Critical Value(10%)'] = t[4]['10%']
print(output)

                                   value
Test Statistic Value            -16.0343
p-value                      6.00936e-29
Lags Used                             17
Number of Observations Used         4259
Critical Value(1%)              -3.43189
Critical Value(5%)              -2.86222
Critical Value(10%)             -2.56713


In [12]:
(p, q) =(sm.tsa.arma_order_select_ic(returns,max_ar=3,max_ma=3,ic='aic')['aic_min_order'])

In [26]:
am = arch_model(returns,mean='AR',lags=3,vol='GARCH') 
res = am.fit()
forecasts = res.forecast(horizon=5)
forecasts.variance.dropna()

Iteration:      1,   Func. Count:      9,   Neg. LLF: 6129.67470532
Iteration:      2,   Func. Count:     22,   Neg. LLF: 6123.15118622
Iteration:      3,   Func. Count:     35,   Neg. LLF: 6122.61946369
Iteration:      4,   Func. Count:     46,   Neg. LLF: 6119.82761469
Iteration:      5,   Func. Count:     59,   Neg. LLF: 6119.74021788
Iteration:      6,   Func. Count:     71,   Neg. LLF: 6119.5617966
Iteration:      7,   Func. Count:     83,   Neg. LLF: 6119.48909348
Iteration:      8,   Func. Count:     93,   Neg. LLF: 6117.93758659
Iteration:      9,   Func. Count:    104,   Neg. LLF: 6117.83725007
Iteration:     10,   Func. Count:    113,   Neg. LLF: 6117.07467283
Iteration:     11,   Func. Count:    122,   Neg. LLF: 6115.81802025
Iteration:     12,   Func. Count:    131,   Neg. LLF: 6115.75621502
Iteration:     13,   Func. Count:    140,   Neg. LLF: 6115.75387949
Iteration:     14,   Func. Count:    149,   Neg. LLF: 6115.753862
Optimization terminated successfully.    (Exit mode

,h.1,h.2,h.3,h.4,h.5
Date,,,,,
2016-12-30,0.404997,0.421208,0.436856,0.451938,0.466716


## Basic Forecasting

Forecasts can be generated for standard GARCH(p,q) processes using any of the three forecast generation methods:

Analytical
Simulation-based
Bootstrap-based
Be default forecasts will only be produced for the final observation in the sample so that they are out-of-sample.

Forecasts start with specifying the model and estimating parameters.

In [23]:
am = arch_model(returns, vol='Garch', p=1, o=0, q=1, dist='Normal')
res = am.fit(update_freq=5)

Iteration:      1,   Func. Count:      6,   Neg. LLF: 6139.78858301
Iteration:      2,   Func. Count:     16,   Neg. LLF: 6136.19371623
Iteration:      3,   Func. Count:     24,   Neg. LLF: 6132.33139526
Iteration:      4,   Func. Count:     32,   Neg. LLF: 6132.1649486
Iteration:      5,   Func. Count:     39,   Neg. LLF: 6130.46329048
Iteration:      6,   Func. Count:     46,   Neg. LLF: 6130.15103208
Iteration:      7,   Func. Count:     53,   Neg. LLF: 6128.71766239
Iteration:      8,   Func. Count:     59,   Neg. LLF: 6128.48693945
Iteration:      9,   Func. Count:     65,   Neg. LLF: 6128.47375042
Iteration:     10,   Func. Count:     71,   Neg. LLF: 6128.47317714
Iteration:     11,   Func. Count:     77,   Neg. LLF: 6128.4731682
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6128.4731682
            Iterations: 11
            Function evaluations: 77
            Gradient evaluations: 11


Forecasts are contained in an ARCHModelForecast object which has 4 attributes:
<ol>
<li>mean - The forecast means</li>
<li>residual_variance - The forecast residual variances, that is Et[ϵ2t+h]Et[ϵt+h2]</li>
<li>variance - The forecast variance of the process, Et[r2t+h]Et[rt+h2]. The variance will differ from the residual variance whenever the model has mean dynamics, e.g., in an AR process.</li>
<li>simulations - An object that contains detailed information about the simulations used to generate forecasts. Only used if the forecast method is set to 'simulation' or 'bootstrap'. If using 'analytical' (the default), this is None.</li>
</ol>
The three main outputs are all returned in DataFrames with columns of the form h.# where # is the number of steps ahead. That is, h.1 corresponds to one-step ahead forecasts while h.10 corresponds to 10-steps ahead.

The default forecast only produces 1-step ahear forecasts.

In [5]:
print(forecasts.mean.iloc[-3:])
print(forecasts.residual_variance.iloc[-3:])
print(forecasts.variance.iloc[-3:])

                 h.1
Date                
2016-12-28       NaN
2016-12-29       NaN
2016-12-30  0.061285
                 h.1
Date                
2016-12-28       NaN
2016-12-29       NaN
2016-12-30  0.400956
                 h.1
Date                
2016-12-28       NaN
2016-12-29       NaN
2016-12-30  0.400956


Longer horizon forecasts can be computed by passing the parameter horizon.

In [9]:
forecasts = res.forecast(horizon=5)
print(forecasts.residual_variance.iloc[-3:])

                 h.1       h.2       h.3       h.4       h.5
Date                                                        
2016-12-28       NaN       NaN       NaN       NaN       NaN
2016-12-29       NaN       NaN       NaN       NaN       NaN
2016-12-30  0.400956  0.416563  0.431896  0.446961  0.461762


Values that are not computed are nan-filled.

## Alternative Forecast Generation Schemes 

### Fixed Window Forecasting ##

Fixed-windows forecasting uses data up to a specified date to generate all forecasts after that date. This cna be implemented by passing the entire data in when initializing the model and then using last_obs when calling fit. forecast() will, by default, produce forecasts after this final date.

Note last_obs follow Python sequence rules so that the actual date in last_obs is not in the sample.

In [10]:
res = am.fit(last_obs = '2011-1-1', update_freq=5)
forecasts = res.forecast(horizon=5)
print(forecasts.variance.dropna().head())

Iteration:      5,   Func. Count:     38,   Neg. LLF: 4204.91956121
Iteration:     10,   Func. Count:     72,   Neg. LLF: 4202.81502485
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 4202.81211069
            Iterations: 12
            Function evaluations: 84
            Gradient evaluations: 12
                 h.1       h.2       h.3       h.4       h.5
Date                                                        
2010-12-31  0.365727  0.376462  0.387106  0.397660  0.408124
2011-01-03  0.451526  0.461532  0.471453  0.481290  0.491043
2011-01-04  0.432131  0.442302  0.452387  0.462386  0.472300
2011-01-05  0.430051  0.440239  0.450341  0.460358  0.470289
2011-01-06  0.407841  0.418219  0.428508  0.438710  0.448825


## Rolling Window Forecasting

Rolling window forecasts use a fixed sample length and then produce one-step from the final observation. These can be implemented using first_obs and last_obs.

In [12]:
index = returns.index
start_loc = 0
end_loc = np.where(index >= '2010-1-1')[0].min()
forecasts = {}
for i in range(20):
    sys.stdout.write('.')
    sys.stdout.flush()
    res = am.fit(first_obs=i, last_obs=i+end_loc, disp='off')
    temp = res.forecast(horizon=3).variance
    fcast = temp.iloc[i+end_loc-1]
    forecasts[fcast.name] = fcast
print()
print(pd.DataFrame(forecasts).T)

....................()
                 h.1       h.2       h.3
2009-12-31  0.598199  0.605960  0.613661
2010-01-04  0.771974  0.778431  0.784837
2010-01-05  0.724185  0.731008  0.737781
2010-01-06  0.674237  0.681423  0.688555
2010-01-07  0.637534  0.644995  0.652399
2010-01-08  0.601684  0.609451  0.617161
2010-01-11  0.562393  0.570450  0.578447
2010-01-12  0.613401  0.621098  0.628738
2010-01-13  0.623059  0.630676  0.638236
2010-01-14  0.584403  0.592291  0.600119
2010-01-15  0.654097  0.661483  0.668813
2010-01-19  0.725471  0.732355  0.739187
2010-01-20  0.758533  0.765176  0.771770
2010-01-21  0.958742  0.964005  0.969229
2010-01-22  1.272999  1.276121  1.279220
2010-01-25  1.182257  1.186084  1.189883
2010-01-26  1.110357  1.114637  1.118885
2010-01-27  1.044077  1.048777  1.053442
2010-01-28  1.085489  1.089873  1.094223
2010-01-29  1.088349  1.092875  1.097367
